In [85]:
import pandas as pd
import requests
from collections import ChainMap
from functools import reduce
import asyncio
import aiohttp
import json
import nest_asyncio
nest_asyncio.apply()

In [2]:
file_path = 'wa_or_zipcode.xlsx'

In [32]:
# Read files
df = pd.read_excel(file_path)
# Convert the columns to integer type

# Access the zip code columns 
or_column = df['OR'].dropna().reset_index(drop=True).astype(int).tolist()
wa_column = df['WA'].dropna().reset_index(drop=True).astype(int).tolist()

In [ ]:
len(or_column), wa_column[0]

In [67]:
def get_uniqu_dealers(dealer_response_list):
    # cover the list dealer to the directionary 
    dealers_dict = {
        dealer['dealer']['id']: {
            'name': dealer['dealer']['name'],
            'phoneNumber': dealer['dealer']['phoneNumber'],
            'siteUrl': dealer['dealer']['siteUrl']
        }
        for dealer in dealer_response_list
    }
    return dealers_dict

(427, 98001)

In [61]:
def get_dealers(zipcode, count):
  """
    1.request the dealer by zipcode
    2. cover the dealer to directionary
  """
    url = f"https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode={zipcode}&count={count}&type=Active"
    print("url=", url)
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        unique_dealer = get_uniqu_dealers(data)
        return unique_dealer
    else:
        print(f"Error fetching data. Status code: {response.status_code}-zipcode{zipcode}")
        return None

In [93]:
async def get_dealers_cocurent(session, zipcode, count):
    url = f"https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode={zipcode}&count={count}&type=Active"
    
    async with session.get(url) as response:
        if response.status == 200:
            data = await response.json()
            unique_dealer = get_uniqu_dealers(data)
            return unique_dealer
        else:
            print(f"Error fetching data. Status code: {response.status_code}-zipcode{zipcode}")
            return None

In [94]:
async def fetch_all_dealers(zip_codes, count):
     async with aiohttp.ClientSession() as session:
        tasks = []
        for zip_code in zip_codes:
            task= get_dealers_cocurent(session, zip_code, count)
            tasks.append(task)
        results = await asyncio.gather(*tasks)
        return results

In [95]:
# Run the asynchronous function
def run_event_loop(zip_codes, count):
    loop = asyncio.get_event_loop()
    combined_results = loop.run_until_complete(fetch_all_dealers(zip_codes, count))
    print("finish")
    return combined_results

In [96]:
wa_dealer = run_event_loop(wa_column, 10)

finish


In [102]:
wa_dealers_unique = get_uniqut_dealer_from_dirc(wa_dealer)

In [147]:
wa_dealer

[{'402316': {'name': "Rairdon's Subaru of Auburn",
   'phoneNumber': '2537371345',
   'siteUrl': 'https://subaruofauburn.com'},
  '140671': {'name': 'Subaru of Puyallup',
   'phoneNumber': '2532865900',
   'siteUrl': 'https://www.subaruofpuyallup.com'},
  '140278': {'name': "Walker's Renton Subaru",
   'phoneNumber': '4252262775',
   'siteUrl': 'https://www.rentonsubaru.com'},
  '140100': {'name': 'Tacoma Subaru',
   'phoneNumber': '2534736200',
   'siteUrl': 'https://tacomasubaru.com'},
  '404253': {'name': "Michael's Subaru of Bellevue",
   'phoneNumber': '4254375452',
   'siteUrl': 'https://www.michaelssubaru.com'},
  '140771': {'name': 'Carter Subaru Ballard',
   'phoneNumber': '2067827475',
   'siteUrl': 'https://www.cartersubaruballard.com/'},
  '140659': {'name': 'Peninsula Subaru',
   'phoneNumber': '3604794320',
   'siteUrl': 'https://www.peninsulasubaru.com'},
  '401136': {'name': 'Eastside Subaru',
   'phoneNumber': '4258208993',
   'siteUrl': 'https://www.eastsidesubaru.com

In [104]:
len(wa_dealers_unique)

36

In [62]:
dealer_info = get_dealers(or_column[1], 10)
dealer_info

url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97002&count=10&type=Active


{'402458': {'name': 'Wilsonville Subaru',
  'phoneNumber': '5037835980',
  'siteUrl': 'https://wilsonvillesubaru.com'},
 '140666': {'name': 'Lithia Subaru - Oregon City',
  'phoneNumber': '5036560612',
  'siteUrl': 'https://www.oregoncitysubaru.com'},
 '140377': {'name': 'Carr Subaru',
  'phoneNumber': '5036442161',
  'siteUrl': 'https://www.carrsubaru.com'},
 '403374': {'name': 'Armstrong Subaru',
  'phoneNumber': '5034720383',
  'siteUrl': 'https://www.armstrongsubaru.com'},
 '402558': {'name': 'Subaru of Portland',
  'phoneNumber': '5032322000',
  'siteUrl': 'https://www.subaruofportland.com/'},
 '140383': {'name': 'Royal Moore Subaru',
  'phoneNumber': '5036405660',
  'siteUrl': 'https://www.royalmooresubaru.net'},
 '140533': {'name': 'Capitol Subaru of Salem',
  'phoneNumber': '5035763300',
  'siteUrl': 'https://www.capitolsubaru.com'},
 '401780': {'name': 'Gresham Subaru',
  'phoneNumber': '5036611200',
  'siteUrl': 'https://www.greshamsubaru.com'},
 '140230': {'name': 'Dick Hann

In [60]:
result = get_uniqu_dealers(dealer_info)
result

{'402458': {'name': 'Wilsonville Subaru',
  'phoneNumber': '5037835980',
  'siteUrl': 'https://wilsonvillesubaru.com'},
 '140666': {'name': 'Lithia Subaru - Oregon City',
  'phoneNumber': '5036560612',
  'siteUrl': 'https://www.oregoncitysubaru.com'},
 '140377': {'name': 'Carr Subaru',
  'phoneNumber': '5036442161',
  'siteUrl': 'https://www.carrsubaru.com'},
 '403374': {'name': 'Armstrong Subaru',
  'phoneNumber': '5034720383',
  'siteUrl': 'https://www.armstrongsubaru.com'},
 '402558': {'name': 'Subaru of Portland',
  'phoneNumber': '5032322000',
  'siteUrl': 'https://www.subaruofportland.com/'},
 '140383': {'name': 'Royal Moore Subaru',
  'phoneNumber': '5036405660',
  'siteUrl': 'https://www.royalmooresubaru.net'},
 '140533': {'name': 'Capitol Subaru of Salem',
  'phoneNumber': '5035763300',
  'siteUrl': 'https://www.capitolsubaru.com'},
 '401780': {'name': 'Gresham Subaru',
  'phoneNumber': '5036611200',
  'siteUrl': 'https://www.greshamsubaru.com'},
 '140230': {'name': 'Dick Hann

In [65]:
dealer_list_res = []
for zip in or_column:
    dealer_list_res.append(get_dealers(zip, 10))

url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97001&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97002&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97003&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97004&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97005&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97006&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97007&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97008&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97009&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97011&count=10&ty

url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97124&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97125&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97127&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97128&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97130&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97131&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97132&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97133&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97134&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97135&count=10&ty

url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97347&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97348&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97350&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97351&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97352&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97355&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97357&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97358&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97360&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97361&count=10&ty

url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97456&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97457&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97458&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97459&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97461&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97462&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97463&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97465&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97466&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97467&count=10&ty

url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97703&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97707&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97709&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97710&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97711&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97712&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97720&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97721&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97722&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97730&count=10&ty

url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97886&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97901&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97902&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97903&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97904&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97905&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97906&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97907&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97908&count=10&type=Active
url= https://www.subaru.com/services/dealers/distances/by/zipcode?zipcode=97909&count=10&ty

In [66]:
len(dealer_list_res)

427

In [71]:
dealer_list_res

[{'401728': {'name': 'Tonkin of the Gorge',
   'phoneNumber': '5412962166',
   'siteUrl': 'https://www.tonkinsubaru.com'},
  '140172': {'name': 'Subaru of Bend',
   'phoneNumber': '5413893031',
   'siteUrl': 'https://www.subaruofbend.com'},
  '401780': {'name': 'Gresham Subaru',
   'phoneNumber': '5036611200',
   'siteUrl': 'https://www.greshamsubaru.com'},
  '140189': {'name': 'Harley Swain Subaru',
   'phoneNumber': '5415679696',
   'siteUrl': 'https://www.harleyswainsubaru.com/'},
  '140666': {'name': 'Lithia Subaru - Oregon City',
   'phoneNumber': '5036560612',
   'siteUrl': 'https://www.oregoncitysubaru.com'},
  '402558': {'name': 'Subaru of Portland',
   'phoneNumber': '5032322000',
   'siteUrl': 'https://www.subaruofportland.com/'},
  '402458': {'name': 'Wilsonville Subaru',
   'phoneNumber': '5037835980',
   'siteUrl': 'https://wilsonvillesubaru.com'},
  '140230': {'name': 'Dick Hannah Subaru',
   'phoneNumber': '3603140505',
   'siteUrl': 'https://dickhannahsubaru.com'},
  '1

In [100]:
# merged_dict = dict(ChainMap(*dealer_list_res))
def get_uniqut_dealer_from_dirc(dict_list):
    merged_dict = reduce(lambda a, b: {**a, **b}, dict_list)
    return merged_dict

In [77]:
len(merged_dict)

38

In [78]:
merged_dict

{'401728': {'name': 'Tonkin of the Gorge',
  'phoneNumber': '5412962166',
  'siteUrl': 'https://www.tonkinsubaru.com'},
 '140172': {'name': 'Subaru of Bend',
  'phoneNumber': '5413893031',
  'siteUrl': 'https://www.subaruofbend.com'},
 '401780': {'name': 'Gresham Subaru',
  'phoneNumber': '5036611200',
  'siteUrl': 'https://www.greshamsubaru.com'},
 '140189': {'name': 'Harley Swain Subaru',
  'phoneNumber': '5415679696',
  'siteUrl': 'https://www.harleyswainsubaru.com/'},
 '140666': {'name': 'Lithia Subaru - Oregon City',
  'phoneNumber': '5036560612',
  'siteUrl': 'https://www.oregoncitysubaru.com'},
 '402558': {'name': 'Subaru of Portland',
  'phoneNumber': '5032322000',
  'siteUrl': 'https://www.subaruofportland.com/'},
 '402458': {'name': 'Wilsonville Subaru',
  'phoneNumber': '5037835980',
  'siteUrl': 'https://wilsonvillesubaru.com'},
 '140230': {'name': 'Dick Hannah Subaru',
  'phoneNumber': '3603140505',
  'siteUrl': 'https://dickhannahsubaru.com'},
 '140377': {'name': 'Carr Su

In [82]:
def make_dealler_dictoexcel(dealer_dic, file_address):
    # convert directionary to excel
    data_list = [{'key': key, **values} for key, values in dealer_dic.items()]
    df = pd.DataFrame(data_list)
    df.to_excel(f'{file_address}.xlsx', index=False, engine='openpyxl')
    print('make it successfully')

In [177]:
def make_excel_fromdf(dfname, file_address):
    df.to_excel(f'{file_address}.xlsx', index=False, engine='openpyxl')
    print('make it successfully')

In [83]:
make_dealler_dictoexcel(merged_dict, '/Users/lei/Desktop/OR_Dealer')

make it successfully


In [105]:
make_dealler_dictoexcel(wa_dealers_unique, '/Users/lei/Desktop/wa_Dealer')

make it successfully


In [148]:
#dealer ids
#wa dealer id
# Read files
df = pd.read_excel("/Users/lei/Desktop/wa_Dealer.xlsx")
# Convert the columns to integer type

# Access the zip code columns 
wa_dealers = df['key'].dropna().reset_index(drop=True).astype(int).tolist()

In [149]:
wa_dealers

[402316,
 140671,
 140278,
 140100,
 404253,
 140771,
 140659,
 401136,
 140193,
 140242,
 404158,
 401900,
 140408,
 140214,
 140276,
 140258,
 140230,
 401780,
 404384,
 401728,
 140383,
 140377,
 402558,
 140666,
 402458,
 403374,
 140533,
 140189,
 140101,
 140596,
 140246,
 400310,
 402123,
 400048,
 401395,
 402176]

In [145]:
def get_dealers_inventory(dealer_id):
    url = f"https://www.subaru.com/services/graphql/retailerinventory?page=0&year=2024&models=IMP&dealerCode={dealer_id}&itemsPerPage=30&sortBy=asc"
    print("url=", url)
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        carlist = data['pagedListWrapper']['items']
        cardic = get_each_dealer_inventory(carlist)
        return cardic
    else:
        print(f"Error fetching data. Status code: {response.status_code}-zipcode{zipcode}")
        return None

In [151]:
async def get_inventory_cocurent(session, dealer_id):
    url = f"https://www.subaru.com/services/graphql/retailerinventory?page=0&year=2024&models=IMP&dealerCode={dealer_id}&itemsPerPage=30&sortBy=asc"
    
    async with session.get(url) as response:
        if response.status == 200:
            data = await response.json()
            carlist = data['pagedListWrapper']['items']
            cardic = get_each_dealer_inventory(carlist)
            return cardic
        else:
            print(f"Error fetching data. Status code: {response.status_code}-dealer_id{dealer_id}")
            return None

In [152]:
async def fetch_all_inventory(dealer_ids):
     async with aiohttp.ClientSession() as session:
        tasks = []
        for dealer_id in dealer_ids:
            task= get_inventory_cocurent(session, dealer_id)
            tasks.append(task)
        results = await asyncio.gather(*tasks)
        return results

In [153]:
def run_event_loop_inventory(dealer_ids):
    loop = asyncio.get_event_loop()
    combined_results = loop.run_until_complete(fetch_all_inventory(dealer_ids))
    print("finish")
    return combined_results

In [155]:
all_cars = run_event_loop_inventory(wa_dealers)

finish


In [146]:
dic = get_dealers_inventory(401136)
dic

url= https://www.subaru.com/services/graphql/retailerinventory?page=0&year=2024&models=IMP&dealerCode=401136&itemsPerPage=30&sortBy=asc


{'JF1GUABC9R8385398': {'dealerCode': '401136',
  'year': 2024,
  'stockNumber': '385398',
  'modelName': 'Impreza',
  'trimCode': 'RLA',
  'trimName': 'Base',
  'msrp': 22995,
  'tsrp': 24652,
  'internetPrice': 24652,
  'detailsUrl': 'http://www.eastsidesubaru.com/catcher.esl?vehicleId=1254973bac1827628aa46e5b7c5ce44c',
  'inTransit': False,
  'destinationFee': 1120},
 'JF1GUAFC9R8331142': {'dealerCode': '401136',
  'year': 2024,
  'stockNumber': '331142',
  'modelName': 'Impreza',
  'trimCode': 'RLD',
  'trimName': 'Sport',
  'msrp': 24995,
  'tsrp': 29161,
  'internetPrice': 29161,
  'detailsUrl': 'http://www.eastsidesubaru.com/catcher.esl?vehicleId=f41caf030a0e081d7752f4ec6a99cc30',
  'inTransit': False,
  'destinationFee': 1120},
 'JF1GUHJC1R8378844': {'dealerCode': '401136',
  'year': 2024,
  'stockNumber': '378844',
  'modelName': 'Impreza',
  'trimCode': 'RLG',
  'trimName': 'RS',
  'msrp': 27885,
  'tsrp': 32647,
  'internetPrice': 32647,
  'detailsUrl': 'http://www.eastsidesu

In [157]:
len(all_cars)

36

In [125]:
cars = get_dealers_inventory(401136)

url= https://www.subaru.com/services/graphql/retailerinventory?page=0&year=2024&models=IMP&dealerCode=401136&itemsPerPage=30&sortBy=asc


In [128]:
cars['pagedListWrapper']['items']

[{'vinNumber': 'JF1GUABC9R8385398',
  'dealerCode': '401136',
  'year': 2024,
  'make': 'Subaru            ',
  'modelName': 'Impreza',
  'trimCode': 'RLA',
  'trimName': 'Base',
  'optionPrice': 0,
  'bodyStyle': '5 Door',
  'transmission': 'CVT',
  'stockNumber': '385398',
  'msrp': 22995,
  'tsrp': 24652,
  'internetPrice': 24652,
  'engine': '2.0-liter SUBARU BOXER<sup>&reg;</sup> 4-cylinder',
  'wheelBase': None,
  'cityMpg': 0,
  'highwayMpg': 0,
  'detailsUrl': 'http://www.eastsidesubaru.com/catcher.esl?vehicleId=1254973bac1827628aa46e5b7c5ce44c',
  'modelCode': 'IMP',
  'exteriorColor': {'code': 'WCH',
   'hydrateValue': 6,
   'name': 'Sapphire Blue Pearl',
   'rgb': [0, 48, 109],
   'default': False,
   'msrp': 0,
   'interiorColors': [{'code': 'BC',
     'hydrateValue': 0,
     'name': 'Black Cloth',
     'rgb': [0, 0, 0],
     'default': False,
     'requiredPackages': [],
     'colorChip': 'https://s7d1.scene7.com/is/image/scom/2024_IMP_BC_color_chip',
     'map': [['RLA']]

In [141]:
def get_each_dealer_inventory(car_list):
    dealers_dict = {}
    for car in car_list:
        dealers_dict.update({
            car['vinNumber']: {
             'dealerCode': car['dealerCode'],
             'year': car['year'],
             'stockNumber':car['stockNumber'],
             'modelName': car['modelName'],
             'trimCode': car['trimCode'],
             'trimName': car['trimName'],
             'msrp': car['msrp'],
             'tsrp': car['tsrp'],
             'internetPrice': car['internetPrice'],
             'detailsUrl':car['detailsUrl'],
             'inTransit': car['inTransit'],
             'destinationFee': car['destinationFee']
            }
        })
    return dealers_dict
    

In [139]:
get_each_dealer_inventory(cars['pagedListWrapper']['items'])

{'JF1GUABC9R8385398': {'dealerCode': '401136',
  'year': 2024,
  'stockNumber': '385398',
  'modelName': 'Impreza',
  'trimCode': 'RLA',
  'trimName': 'Base',
  'msrp': 22995,
  'tsrp': 24652,
  'internetPrice': 24652,
  'detailsUrl': 'http://www.eastsidesubaru.com/catcher.esl?vehicleId=1254973bac1827628aa46e5b7c5ce44c',
  'inTransit': False,
  'destinationFee': 1120},
 'JF1GUAFC9R8331142': {'dealerCode': '401136',
  'year': 2024,
  'stockNumber': '331142',
  'modelName': 'Impreza',
  'trimCode': 'RLD',
  'trimName': 'Sport',
  'msrp': 24995,
  'tsrp': 29161,
  'internetPrice': 29161,
  'detailsUrl': 'http://www.eastsidesubaru.com/catcher.esl?vehicleId=f41caf030a0e081d7752f4ec6a99cc30',
  'inTransit': False,
  'destinationFee': 1120},
 'JF1GUHJC1R8378844': {'dealerCode': '401136',
  'year': 2024,
  'stockNumber': '378844',
  'modelName': 'Impreza',
  'trimCode': 'RLG',
  'trimName': 'RS',
  'msrp': 27885,
  'tsrp': 32647,
  'internetPrice': 32647,
  'detailsUrl': 'http://www.eastsidesu

In [164]:
all_cars

[{'JF1GUABCXR8391727': {'dealerCode': '402316',
   'year': 2024,
   'stockNumber': None,
   'modelName': 'Impreza',
   'trimCode': 'RLA',
   'trimName': 'Base',
   'msrp': 22995,
   'tsrp': 24465,
   'internetPrice': 24465,
   'detailsUrl': 'https://www.subaruofauburn.com/inventory/new-2024-subaru-impreza-base-all-wheel-drive-5-door-jf1guabcxr8391727',
   'inTransit': True,
   'destinationFee': 1120},
  'JF1GUAFCXR8371651': {'dealerCode': '402316',
   'year': 2024,
   'stockNumber': 'R8371651',
   'modelName': 'Impreza',
   'trimCode': 'RLD',
   'trimName': 'Sport',
   'msrp': 24995,
   'tsrp': 28598,
   'internetPrice': 26825,
   'detailsUrl': 'https://www.subaruofauburn.com/inventory/new-2024-subaru-impreza-sport-awd-5-door-jf1guafcxr8371651/',
   'inTransit': False,
   'destinationFee': 1120},
  'JF1GUHJC6R8364549': {'dealerCode': '402316',
   'year': 2024,
   'stockNumber': 'R8364549',
   'modelName': 'Impreza',
   'trimCode': 'RLG',
   'trimName': 'RS',
   'msrp': 27885,
   'tsrp'

In [160]:
wa_inventory = get_uniqut_dealer_from_dirc(all_cars)
wa_inventory


{'JF1GUABCXR8391727': {'dealerCode': '402316',
  'year': 2024,
  'stockNumber': None,
  'modelName': 'Impreza',
  'trimCode': 'RLA',
  'trimName': 'Base',
  'msrp': 22995,
  'tsrp': 24465,
  'internetPrice': 24465,
  'detailsUrl': 'https://www.subaruofauburn.com/inventory/new-2024-subaru-impreza-base-all-wheel-drive-5-door-jf1guabcxr8391727',
  'inTransit': True,
  'destinationFee': 1120},
 'JF1GUAFCXR8371651': {'dealerCode': '402316',
  'year': 2024,
  'stockNumber': 'R8371651',
  'modelName': 'Impreza',
  'trimCode': 'RLD',
  'trimName': 'Sport',
  'msrp': 24995,
  'tsrp': 28598,
  'internetPrice': 26825,
  'detailsUrl': 'https://www.subaruofauburn.com/inventory/new-2024-subaru-impreza-sport-awd-5-door-jf1guafcxr8371651/',
  'inTransit': False,
  'destinationFee': 1120},
 'JF1GUHJC6R8364549': {'dealerCode': '402316',
  'year': 2024,
  'stockNumber': 'R8364549',
  'modelName': 'Impreza',
  'trimCode': 'RLG',
  'trimName': 'RS',
  'msrp': 27885,
  'tsrp': 32353,
  'internetPrice': 3016

In [161]:
len(wa_inventory)

130

In [165]:
wa_inventory

{'JF1GUABCXR8391727': {'dealerCode': '402316',
  'year': 2024,
  'stockNumber': None,
  'modelName': 'Impreza',
  'trimCode': 'RLA',
  'trimName': 'Base',
  'msrp': 22995,
  'tsrp': 24465,
  'internetPrice': 24465,
  'detailsUrl': 'https://www.subaruofauburn.com/inventory/new-2024-subaru-impreza-base-all-wheel-drive-5-door-jf1guabcxr8391727',
  'inTransit': True,
  'destinationFee': 1120},
 'JF1GUAFCXR8371651': {'dealerCode': '402316',
  'year': 2024,
  'stockNumber': 'R8371651',
  'modelName': 'Impreza',
  'trimCode': 'RLD',
  'trimName': 'Sport',
  'msrp': 24995,
  'tsrp': 28598,
  'internetPrice': 26825,
  'detailsUrl': 'https://www.subaruofauburn.com/inventory/new-2024-subaru-impreza-sport-awd-5-door-jf1guafcxr8371651/',
  'inTransit': False,
  'destinationFee': 1120},
 'JF1GUHJC6R8364549': {'dealerCode': '402316',
  'year': 2024,
  'stockNumber': 'R8364549',
  'modelName': 'Impreza',
  'trimCode': 'RLG',
  'trimName': 'RS',
  'msrp': 27885,
  'tsrp': 32353,
  'internetPrice': 3016

In [166]:
make_dealler_dictoexcel(wa_inventory, '/Users/lei/Desktop/wa_inventory')

make it successfully


In [169]:
# Read files
df_wa_inventory = pd.read_excel("/Users/lei/Desktop/wa_inventory.xlsx")

In [170]:
# make aggregation to know the car situation
grouped_df = df_wa_inventory.groupby('dealerCode').size().reset_index(name='carCount')


In [171]:
grouped_df

,dealerCode,carCount
0,140100,14
1,140101,6
2,140189,1
3,140193,4
4,140230,5
5,140242,4
6,140246,3
7,140258,1
8,140276,4
9,140278,5


In [172]:
df_wa_dealer= pd.read_excel("/Users/lei/Desktop/wa_Dealer.xlsx")

In [173]:
df_wa_dealer

,key,name,phoneNumber,siteUrl
0,402316,Rairdon's Subaru of Auburn,2537371345,https://subaruofauburn.com
1,140671,Subaru of Puyallup,2532865900,https://www.subaruofpuyallup.com
2,140278,Walker's Renton Subaru,4252262775,https://www.rentonsubaru.com
3,140100,Tacoma Subaru,2534736200,https://tacomasubaru.com
4,404253,Michael's Subaru of Bellevue,4254375452,https://www.michaelssubaru.com
5,140771,Carter Subaru Ballard,2067827475,https://www.cartersubaruballard.com/
6,140659,Peninsula Subaru,3604794320,https://www.peninsulasubaru.com
7,401136,Eastside Subaru,4258208993,https://www.eastsidesubaru.com
8,140193,Carter Subaru,2065421166,https://www.cartersubarushoreline.com/
9,140242,Hanson Subaru,3609432120,https://www.hansonsubaru.com/


In [174]:
wa_dealer_intentory = pd.merge(grouped_df, df_wa_dealer,  left_on='dealerCode', right_on='key', how='left')

In [182]:
wa_dealer_intentory

,dealerCode,carCount,key,name,phoneNumber,siteUrl
0,140100,14,140100,Tacoma Subaru,2534736200,https://tacomasubaru.com
1,140101,6,140101,McCurley Subaru,5094127100,https://www.mccurleysubaru.com
2,140189,1,140189,Harley Swain Subaru,5415679696,https://www.harleyswainsubaru.com/
3,140193,4,140193,Carter Subaru,2065421166,https://www.cartersubarushoreline.com/
4,140230,5,140230,Dick Hannah Subaru,3603140505,https://dickhannahsubaru.com
5,140242,4,140242,Hanson Subaru,3609432120,https://www.hansonsubaru.com/
6,140246,3,140246,AutoNation Subaru Spokane Valley,5098509358,https://www.autonationsubaruspokane.com
7,140258,1,140258,Bud Clary Subaru,3604231700,https://budclarysubaru.com
8,140276,4,140276,Dewey Griffin Subaru,3607348700,https://www.deweygriffinsubaru.com/
9,140278,5,140278,Walker's Renton Subaru,4252262775,https://www.rentonsubaru.com


In [181]:
wa_dealer_intentory.to_excel('/Users/lei/Desktop/wa_Dealer_intentory.xlsx', index=False, engine='openpyxl')

In [187]:
di_columns = wa_dealer_intentory.columns

In [188]:
merged_df = wa_dealer_intentory[di_columns]

In [190]:
merged_df.to_excel('/Users/lei/Desktop/wa_Dealer_intentory.xlsx', index=False, engine='openpyxl')